In [20]:
import http.client
import json
import time
import csv
from datetime import datetime
from datetime import timedelta
import numpy as np
import pandas as pd
import re
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO


class NRGStreamApi:    
    
    def __init__(self,username=None,password=None):
        self.username = 'Fahad'
        self.password = 'ABFAHc2'                
        self.server = 'api.nrgstream.com'        
        self.tokenPath = '/api/security/token'
        self.releasePath = '/api/ReleaseToken'
        self.tokenPayload = f'grant_type=password&username={self.username}&password={self.password}'
        self.tokenExpiry = datetime.now() - timedelta(seconds=60)
        self.accessToken = ""        
        
    def getToken(self):
        try:
            if self.isTokenValid() == False:                             
                headers = { }        
                # Connect to API server to get a token
                conn = http.client.HTTPSConnection(self.server)
                conn.request('POST', self.tokenPath, self.tokenPayload, headers)
                res = conn.getresponse()                
                res_code = res.code
                # Check if the response is good
                
                if res_code == 200:
                    res_data = res.read()
                    # Decode the token into an object
                    jsonData = json.loads(res_data.decode('utf-8'))
                    self.accessToken = jsonData['access_token']                         
                    # Calculate new expiry date
                    self.tokenExpiry = datetime.now() + timedelta(seconds=jsonData['expires_in'])                        
                    #print('token obtained')
                    #print(self.accessToken)
                else:
                    res_data = res.read()
                    print(res_data.decode('utf-8'))
                conn.close()                          
        except Exception as e:
            print("getToken: " + str(e))
            # Release token if an error occured
            self.releaseToken()      

    def releaseToken(self):
        try:            
            headers = {}
            headers['Authorization'] = f'Bearer {self.accessToken}'            
            conn = http.client.HTTPSConnection(self.server)
            conn.request('DELETE', self.releasePath, None, headers)  
            res = conn.getresponse()
            res_code = res.code
            if res_code == 200:   
                # Set expiration date back to guarantee isTokenValid() returns false                
                self.tokenExpiry = datetime.now() - timedelta(seconds=60)
                #print('token released')            
        except Exception as e:
            print("releaseToken: " + str(e))
                    
    def isTokenValid(self):
        if self.tokenExpiry==None:
            return False
        elif datetime.now() >= self.tokenExpiry:            
            return False
        else:
            return True            
    
    def GetStreamDataByStreamId(self,streamIds, fromDate, toDate, dataFormat='csv', dataOption=''):
        stream_data = "" 
        # Set file format to csv or json            
        DataFormats = {}
        DataFormats['csv'] = 'text/csv'
        DataFormats['json'] = 'Application/json'
        
        try:                            
            for streamId in streamIds:            
                # Get an access token            
                self.getToken()    
                if self.isTokenValid():
                    # Setup the path for data request. Pass dates in via function call
                    path = f'/api/StreamData/{streamId}'
                    if fromDate != '' and toDate != '':
                        path += f'?fromDate={fromDate.replace(" ", "%20")}&toDate={toDate.replace(" ", "%20")}'
                    if dataOption != '':
                        if fromDate != '' and toDate != '':
                            path += f'&dataOption={dataOption}'        
                        else:
                            path += f'?dataOption={dataOption}'        
                    
                    # Create request header
                    headers = {}            
                    headers['Accept'] = DataFormats[dataFormat]
                    headers['Authorization']= f'Bearer {self.accessToken}'
                    
                    # Connect to API server
                    conn = http.client.HTTPSConnection(self.server)
                    conn.request('GET', path, None, headers)
                    res = conn.getresponse()        
                    res_code = res.code                    
                    if res_code == 200:   
                        try:
                            print(f'{datetime.now()} Outputing stream {path} res code {res_code}')
                            # output return data to a text file            
                            if dataFormat == 'csv':
                                stream_data += res.read().decode('utf-8').replace('\r\n','\n') 
                            elif dataFormat == 'json':
                                stream_data += json.dumps(json.loads(res.read().decode('utf-8')), indent=2, sort_keys=False)
                            conn.close()

                        except Exception as e:
                            print(str(e))            
                            self.releaseToken()
                            return None  
                    else:
                        print(str(res_code) + " - " + str(res.reason) + " - " + str(res.read().decode('utf-8')))
                    
                self.releaseToken()   
                # Wait 1 second before next request
                time.sleep(1)
            return stream_data        
        except Exception as e:
            print(str(e))    
            self.releaseToken()
            return None
        
        
    def StreamDataOptions(self, streamId, dataFormat='csv'):
        try:      
            DataFormats = {}
            DataFormats['csv'] = 'text/csv'
            DataFormats['json'] = 'Application/json'
            resultSet = {}
            for streamId in streamIds:
                # Get an access token    
                if streamId not in resultSet:
                    self.getToken()                        
                    if self.isTokenValid():                 
                        # Setup the path for data request.
                        path = f'/api/StreamDataOptions/{streamId}'                        
                        # Create request header
                        headers = {}     
                        headers['Accept'] = DataFormats[dataFormat]                                   
                        headers['Authorization'] = f'Bearer {self.accessToken}'
                        # Connect to API server
                        conn = http.client.HTTPSConnection(self.server)
                        conn.request('GET', path, None, headers)
                        res = conn.getresponse()
                        self.releaseToken()       
                        if dataFormat == 'csv':
                            resultSet[streamId] = res.read().decode('utf-8').replace('\r\n','\n') 
                        elif dataFormat == 'json':
                            resultSet[streamId] = json.dumps(json.loads(res.read().decode('utf-8')), indent=2, sort_keys=False)                            
                    time.sleep(1)                        
            return resultSet            
        except Exception as e:
            print(str(e))    
            self.releaseToken()
            return None          
        
        except Exception as e:            
            self.releaseToken()                        
            return str(e)

        
# Authenticate with your NRG Stream username and password    
nrgStreamApi = NRGStreamApi('Username','Password')         
# Date range for your data request
# Date format must be mm/dd/yyyy
fromDate = '01/01/2019'
toDate =   '02/14/2021'
# Specify output format - 'csv' or 'json'
dataFormat = 'csv'
# Data Option
dataOption = ''
stream = [3]
for i in stream:
    nrgStreamApi = NRGStreamApi('Username','Password')
    ids= [i]
    stream_data = nrgStreamApi.GetStreamDataByStreamId(ids, fromDate, toDate, dataFormat, dataOption)        
    STREAM_DATA = StringIO(stream_data)
    df = pd.read_csv(STREAM_DATA, sep=";")

    
df = df[14:df.shape[0]]
df.columns = ["Datetime,AIL"]
new = df['Datetime,AIL'].str.split(",", n = 2, expand = True) 
# making separate first name column from new data frame 
df["Datetime"]= new[0] 
# making separate last name column from new data frame 
df["AIL"]= new[1] 
df['AIL'] = pd.to_numeric(df['AIL'],errors='coerce')
df['Datetime']= pd.to_datetime(df['Datetime'])
df= df.drop(columns=['Datetime,AIL'],axis=1)
df = df.reset_index(drop=True)
df

2021-02-15 20:12:33.169871 Outputing stream /api/StreamData/3?fromDate=01/01/2019&toDate=02/14/2021 res code 200


,Datetime,AIL
0,2019-01-01 00:00:00,10008.0
1,2019-01-01 01:00:00,9868.0
2,2019-01-01 02:00:00,9736.0
3,2019-01-01 03:00:00,9597.0
4,2019-01-01 04:00:00,9530.0
...,...,...
18595,2021-02-13 19:00:00,11312.0
18596,2021-02-13 20:00:00,11228.0
18597,2021-02-13 21:00:00,11092.0
18598,2021-02-13 22:00:00,10939.0


In [21]:
df['just_date'] = df['Datetime'].dt.date
df['year'] = df['Datetime'].dt.year
df['day_of_month'] = df['Datetime'].dt.day
df['month'] = df['Datetime'].dt.month
df['numeric_datetime'] = 10000*df['year'] + 100*df['month'] + df['day_of_month']   

dates = df['just_date']
day = pd.Series([d.timetuple().tm_yday for d in dates])

df['day_of_year'] = day
df['hour_of_day']= df['Datetime'].dt.hour
df['hour_of_the_week'] = df['Datetime'].dt.dayofweek * 24 + (df['Datetime'].dt.hour + 1)

df['hour_x_day'] = df['day_of_year']*df['hour_of_day'] 

#------------------------

df['sin.hour.daily']  = np.sin(df['hour_of_day']*2*np.pi/24)
df['cos.hour.daily']  = np.cos(df['hour_of_day']*2*np.pi/24)

df['sin.hour.weekly'] = np.sin(df['hour_of_the_week']*2*np.pi/168 )
df['cos.hour.weekly'] = np.cos(df['hour_of_the_week']*2*np.pi/168 )

df['sin.day.yearly'] = np.sin(df['day_of_year']*2*np.pi/365 + 3*np.pi/2)
df['cos.day.yearly'] = np.cos(df['day_of_year']*2*np.pi/365 + 3*np.pi/2)

#-------------------------

weekdays = [d.weekday() for d in dates]
df['weekend'] = [1 if d >= 5 else 0 for d in weekdays]

#-----------------------------------------


for i, s in enumerate(['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday',
            'sunday']):
    df[s] = [1 if d == i else 0 for d in weekdays]

    
df

,Datetime,AIL,just_date,year,day_of_month,month,numeric_datetime,day_of_year,hour_of_day,hour_of_the_week,...,sin.day.yearly,cos.day.yearly,weekend,monday,tuesday,wednesday,thursday,friday,saturday,sunday
0,2019-01-01 00:00:00,10008.0,2019-01-01,2019,1,1,20190101,1,0,25,...,-0.999852,0.017213,0,0,1,0,0,0,0,0
1,2019-01-01 01:00:00,9868.0,2019-01-01,2019,1,1,20190101,1,1,26,...,-0.999852,0.017213,0,0,1,0,0,0,0,0
2,2019-01-01 02:00:00,9736.0,2019-01-01,2019,1,1,20190101,1,2,27,...,-0.999852,0.017213,0,0,1,0,0,0,0,0
3,2019-01-01 03:00:00,9597.0,2019-01-01,2019,1,1,20190101,1,3,28,...,-0.999852,0.017213,0,0,1,0,0,0,0,0
4,2019-01-01 04:00:00,9530.0,2019-01-01,2019,1,1,20190101,1,4,29,...,-0.999852,0.017213,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18595,2021-02-13 19:00:00,11312.0,2021-02-13,2021,13,2,20210213,44,19,140,...,-0.726608,0.687053,1,0,0,0,0,0,1,0
18596,2021-02-13 20:00:00,11228.0,2021-02-13,2021,13,2,20210213,44,20,141,...,-0.726608,0.687053,1,0,0,0,0,0,1,0
18597,2021-02-13 21:00:00,11092.0,2021-02-13,2021,13,2,20210213,44,21,142,...,-0.726608,0.687053,1,0,0,0,0,0,1,0
18598,2021-02-13 22:00:00,10939.0,2021-02-13,2021,13,2,20210213,44,22,143,...,-0.726608,0.687053,1,0,0,0,0,0,1,0


In [22]:
df.dtypes

Datetime            datetime64[ns]
AIL                        float64
just_date                   object
year                         int64
day_of_month                 int64
month                        int64
numeric_datetime             int64
day_of_year                  int64
hour_of_day                  int64
hour_of_the_week             int64
hour_x_day                   int64
sin.hour.daily             float64
cos.hour.daily             float64
sin.hour.weekly            float64
cos.hour.weekly            float64
sin.day.yearly             float64
cos.day.yearly             float64
weekend                      int64
monday                       int64
tuesday                      int64
wednesday                    int64
thursday                     int64
friday                       int64
saturday                     int64
sunday                       int64
dtype: object

In [23]:
df1 = pd.DataFrame(df)

In [24]:
fromDate ='01/01/2019'
toDate   ='02/15/2021'

df_clgry   = pd.DataFrame()
df_edmtn   = pd.DataFrame()
df_list = {}

streams = [242498, 242497]
city_names = ['calgary', 'edmonton']
k = 0

for i in streams:
    stream_data = nrgStreamApi.GetStreamDataByStreamId([i], fromDate, toDate, 'csv', '')        
    STREAM_DATA = StringIO(stream_data)
    df = pd.read_csv(STREAM_DATA, sep=";")
    length = df.shape[0]
    df = df[16:length] # removing header information
    df.columns = ["Datetime,temp,wind,direction"]
    new = df['Datetime,temp,wind,direction'].str.split(",", n = 4, expand = True) 
    # making separate first name column from new data frame 
    df["Datetime"]= new[0] 
    df["temp"]= new[1] 
    df["wind"] = new [2]
    df["direction"] = new[3]
    df = df.drop(['Datetime,temp,wind,direction','direction', "wind"],axis=1)
    df = df.reset_index(drop=True)
    df.columns = [str(col) + '_'+ city_names[k] for col in df.columns]
    df_list[k] = df
    k = k+1

df_clgry   = df_list[0]
df_edmtn   = df_list[1]


alberta_weather_merged = pd.concat([df_clgry, df_edmtn], axis=1)

alberta_weather_merged = alberta_weather_merged[['Datetime_calgary', 'temp_calgary', 'temp_edmonton']]

#-----

fromDate ='01/01/2019'
toDate   ='05/22/2019'

df_clgry   = pd.DataFrame()
df_edmtn   = pd.DataFrame()

df_list = {}

streams = [5, 11]
city_names = ['calgary', 'edmonton']
k = 0

for i in streams:
    stream_data = nrgStreamApi.GetStreamDataByStreamId([i], fromDate, toDate, 'csv', '')        
    STREAM_DATA = StringIO(stream_data)
    df = pd.read_csv(STREAM_DATA, sep=";")
    length = df.shape[0]
    df = df[16:length] # removing header information
    df.columns = ["Datetime,temp,wind,direction"]
    new = df['Datetime,temp,wind,direction'].str.split(",", n = 4, expand = True) 
    # making separate first name column from new data frame 
    df["Datetime"]= new[0] 
    df["temp"]= new[1] 
    df["wind"] = new [2]
    df["direction"] = new[3]
    df = df.drop(['Datetime,temp,wind,direction','direction','wind'],axis=1)
    df = df.reset_index(drop=True)
    df.columns = [str(col) + '_'+ city_names[k] for col in df.columns]
    df_list[k] = df
    k = k+1

df_clgry   = df_list[0]
df_edmtn   = df_list[1]



alberta_weather_merged_2 = pd.concat([df_clgry, df_edmtn], axis=1)
alberta_weather_merged_2 = alberta_weather_merged_2[['Datetime_calgary', 'temp_calgary', 'temp_edmonton']]
#-----
alberta_weather_merged_2         = alberta_weather_merged_2.append(alberta_weather_merged)
alberta_weather_merged_2.columns =['Datetime', 'temp_calgary', 'temp_edmonton']

alberta_weather_merged_2['Datetime'] = pd.to_datetime(alberta_weather_merged_2['Datetime'])
alberta_weather_merged_2['year_wt']         = alberta_weather_merged_2['Datetime'].dt.year
alberta_weather_merged_2['day_of_month_wt'] = alberta_weather_merged_2['Datetime'].dt.day
alberta_weather_merged_2['month_wt']        = alberta_weather_merged_2['Datetime'].dt.month
alberta_weather_merged_2['numeric_datetime'] = 10000*alberta_weather_merged_2['year_wt'] + 100*alberta_weather_merged_2['month_wt'] + alberta_weather_merged_2['day_of_month_wt']   
alberta_weather_merged_2['temp_calgary'] = pd.to_numeric(alberta_weather_merged_2['temp_calgary'])
alberta_weather_merged_2['temp_edmonton'] = pd.to_numeric(alberta_weather_merged_2['temp_edmonton'])

reduced_columns = ['numeric_datetime', 'temp_calgary', 'temp_edmonton']
alberta_weather_merged_2 = pd.DataFrame(data = alberta_weather_merged_2, columns = reduced_columns)
alberta_weather_merged_2

2021-02-15 20:16:01.827545 Outputing stream /api/StreamData/242498?fromDate=01/01/2019&toDate=02/15/2021 res code 200
2021-02-15 20:16:04.900589 Outputing stream /api/StreamData/242497?fromDate=01/01/2019&toDate=02/15/2021 res code 200
2021-02-15 20:16:07.460241 Outputing stream /api/StreamData/5?fromDate=01/01/2019&toDate=05/22/2019 res code 200
2021-02-15 20:16:10.123098 Outputing stream /api/StreamData/11?fromDate=01/01/2019&toDate=05/22/2019 res code 200


,numeric_datetime,temp_calgary,temp_edmonton
0,20190101,-11.1,-9.8
1,20190101,-8.7,-9.3
2,20190101,-7.4,-8.2
3,20190101,-4.9,-7.3
4,20190101,-0.9,-6.3
...,...,...,...
15235,20210214,-22.0,-19.0
15236,20210214,-23.0,-19.0
15237,20210214,-23.0,-23.0
15238,20210214,-23.0,-23.0


In [25]:
alberta_weather_merged_2.dtypes

numeric_datetime      int64
temp_calgary        float64
temp_edmonton       float64
dtype: object

In [43]:
df_all = pd.merge(df1, alberta_weather_merged_2, on="numeric_datetime")
df_all = df_all.drop(columns=['numeric_datetime'])
df_all = df_all.drop_duplicates(subset=['Datetime'])
df_all = df_all.reset_index(drop= True)
df_all

,Datetime,AIL,just_date,year,day_of_month,month,day_of_year,hour_of_day,hour_of_the_week,hour_x_day,...,weekend,monday,tuesday,wednesday,thursday,friday,saturday,sunday,temp_calgary,temp_edmonton
0,2019-01-01 00:00:00,10008.0,2019-01-01,2019,1,1,1,0,25,0,...,0,0,1,0,0,0,0,0,-11.1,-9.8
1,2019-01-01 01:00:00,9868.0,2019-01-01,2019,1,1,1,1,26,1,...,0,0,1,0,0,0,0,0,-11.1,-9.8
2,2019-01-01 02:00:00,9736.0,2019-01-01,2019,1,1,1,2,27,2,...,0,0,1,0,0,0,0,0,-11.1,-9.8
3,2019-01-01 03:00:00,9597.0,2019-01-01,2019,1,1,1,3,28,3,...,0,0,1,0,0,0,0,0,-11.1,-9.8
4,2019-01-01 04:00:00,9530.0,2019-01-01,2019,1,1,1,4,29,4,...,0,0,1,0,0,0,0,0,-11.1,-9.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18595,2021-02-13 19:00:00,11312.0,2021-02-13,2021,13,2,44,19,140,836,...,1,0,0,0,0,0,1,0,-21.0,-20.0
18596,2021-02-13 20:00:00,11228.0,2021-02-13,2021,13,2,44,20,141,880,...,1,0,0,0,0,0,1,0,-21.0,-20.0
18597,2021-02-13 21:00:00,11092.0,2021-02-13,2021,13,2,44,21,142,924,...,1,0,0,0,0,0,1,0,-21.0,-20.0
18598,2021-02-13 22:00:00,10939.0,2021-02-13,2021,13,2,44,22,143,968,...,1,0,0,0,0,0,1,0,-21.0,-20.0


In [44]:
df_all.columns

Index(['Datetime', 'AIL', 'just_date', 'year', 'day_of_month', 'month',
       'day_of_year', 'hour_of_day', 'hour_of_the_week', 'hour_x_day',
       'sin.hour.daily', 'cos.hour.daily', 'sin.hour.weekly',
       'cos.hour.weekly', 'sin.day.yearly', 'cos.day.yearly', 'weekend',
       'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday',
       'sunday', 'temp_calgary', 'temp_edmonton'],
      dtype='object')

In [46]:
df_all.to_csv('training_data_15feb2021.csv')